In [11]:
from generator import *

In [12]:
A = LFSR(np.array([1, 1]), np.array([1, 1]))
B = LFSR(np.array([0, 1, 0]), np.array([1, 1, 0]))
C = LFSR(np.array([0, 1, 0, 1]), np.array([1, 0, 0, 1]))
D = LFSR(np.array([1, 1, 0, 1, 0, 1, 0]), np.array([1, 0, 1, 0, 0, 1, 1]))
projection_matrix = np.array([[1, 0, 0, 0], 
                              [0, 0, 0, 0],
                              [0, 0, 0, 0],
                              [0, 1, 0, 0],
                              [0, 0, 0, 0],
                              [0, 0, 0, 0],
                              [0, 0, 0, 0],
                              [0, 0, 0, 0],
                              [0, 0, 1, 0],
                              [0, 0, 0, 0],
                              [0, 0, 0, 0],
                              [0, 0, 0, 0],
                              [0, 0, 0, 0],
                              [0, 0, 0, 0],
                              [0, 0, 0, 1],
                              [0, 0, 0, 0]])
combiner = LMcombiner([A, B, C, D], projection_matrix, lambda at, bt, ct, dt: at * dt + bt * ct + ct * dt + at * bt)

keystream = []
print(combiner.calculate_output())
keystream.append(combiner.calculate_output())
for i in range(80):
    keystream.append(combiner.next())
keystream

0


[0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0]

In [13]:
key_symbols = sp.symbols("a:16")
A = LFSR(np.array(key_symbols[:2]), np.array([1, 1]))
B = LFSR(np.array(key_symbols[2:5]), np.array([1, 1, 0]))
C = LFSR(np.array(key_symbols[5:9]), np.array([1, 0, 0, 1]))
D = LFSR(np.array(key_symbols[9:]), np.array([1, 0, 1, 0, 0, 1, 1]))
combiner = SymLMcombiner([A, B, C, D], projection_matrix, lambda at, bt, ct, dt: at * dt + bt * ct + ct * dt + at * bt)

sys = []

symbols = combiner.calculate_output().free_symbols
sys.append(sp.Eq(combiner.calculate_output(), keystream[0]))
for i in range(1, 80):
    poly = combiner.next()
    symbols |= poly.free_symbols
    sys.append(sp.Eq(poly, keystream[i]))
symbols = list(symbols)


TypeError: SymLMcombiner.get_monomials() takes 2 positional arguments but 3 were given

In [ ]:
sys_matrices = sp.linear_eq_to_matrix(sys, symbols)
F_2 = sp.GF(2)
a = sp.polys.matrices.domainmatrix.DomainMatrix.from_Matrix(sys_matrices[0]).convert_to(F_2)
b = sp.polys.matrices.domainmatrix.DomainMatrix.from_Matrix(sys_matrices[1]).convert_to(F_2)
solution = a.lu_solve(b).to_Matrix()

sym_values = {symbols[i]: solution[i] for i in range(len(symbols))}
sym_dict = {v: k for k, v in combiner.monomials.items()} | {sym.name: sym for sym in key_symbols}
new_sys = [sp.Eq(sym_dict[sym.name], value) for sym, value in sym_values.items()]
new_sys2 = []

for equation in new_sys:
    if (equation.rhs):
        for sym in equation.lhs.free_symbols:
            if (sp.Eq(sym, 1) not in new_sys2):
                new_sys2.append(sp.Eq(sym, 1))
    else:
        new_sys2.append(equation)
sp.nonlinsolve(new_sys2, key_symbols)

{(1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0)}